In [1]:
import os
import json
import time
from tqdm.notebook import tqdm
import requests
import base64
from io import BytesIO
from PIL import Image


In [2]:
#получаем iam_token
def get_llm_token(token):
    r = requests.post('https://iam.api.cloud.yandex.net/iam/v1/tokens', 
              headers = {'Content-Type': 'application/x-www-form-urlencoded'}, 
              data = '{\"yandexPassportOauthToken\":\"'+token+'\"}').text
    return json.loads(r)["iamToken"]

iam_token = get_llm_token(os.environ['model_oauth_token'])

In [3]:
#список городов
cities = ["Краснодар", 
          "Ставрополь", 
          "Элиста", 
          "Волгоград"]

In [4]:
def photo_city_generate(city, output_folder = "images"):
    prompt = f"Нарисуй реалистичное узнаваемое изображение вида города {city} в России, включая его достопримечательности, для использования на сайте"
    yaart_request = requests.post(
            url ='https://llm.api.cloud.yandex.net/foundationModels/v1/imageGenerationAsync',
            headers = 
                {"Authorization": f"Bearer {iam_token}",
                "x-folder-id":os.environ['model_dir_id']},
            json =
                {"modelUri":"art://"+os.environ['model_dir_id']+"/yandex-art/latest",
                 "messages":[{"text": prompt,
                              "weight":1}],
                 "generationOptions":{"mimeType":"image/jpeg","seed":4900090131875000,
                                      "aspectRatio": {
      "widthRatio": "1400",
      "heightRatio": "400"
    }}}
)
    id_operation = json.loads(yaart_request.text)['id']
    #print(id_operation)
    time.sleep(180)
    yaart_request2 = requests.get(
            url ='https://llm.api.cloud.yandex.net:443/operations/' + id_operation,
            headers = 
                {"Authorization": f"Bearer {iam_token}",
                "x-folder-id":os.environ['model_dir_id']})
    yaart_r2 = json.loads(yaart_request2.text)['response']['image']
    #print(yaart_r2[0:100])
    
    # Декодируем строку base64 в байты
    image_data = base64.b64decode(yaart_r2)
    image = Image.open(BytesIO(image_data))
    # Создаем папку, если не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Полный путь к файлу
    output_path = os.path.join(output_folder, city + ".jpg")
    
    # Сохраняем изображение в папку
    image.save(output_path, format="JPEG")
    image.show()

    
    
photo_city_generate("Москва")
    

In [5]:
for city in tqdm(cities, desc="Processing", unit="iter"):
    photo_city_generate(city)

Processing:   0%|          | 0/4 [00:00<?, ?iter/s]

/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpmgt0b3k0.PNG'
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmp3n37xrxp.PNG'
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpq6g5z3c6.PNG'
